In [ ]:
import fastplotlib as fpl
import torch
import numpy as np
import zmq

# Set up zmq as client subscriber

In [ ]:
context = zmq.Context()
sub = context.socket(zmq.SUB)
sub.setsockopt(zmq.SUBSCRIBE, b"")

# keep only the most recent message
sub.setsockopt(zmq.CONFLATE, 1)

# address must match publisher in Processor actor
sub.connect("tcp://127.0.0.1:5555")

In [ ]:
def get_buffer():
    """Gets the buffer from the publisher."""
    try:
        b = sub.recv(zmq.NOBLOCK)
    except zmq.Again:
        pass
    else:
        return b
    
    return None

In [ ]:
# Create the figure
figure = fpl.Figure(names=[["conv1 weights"]], size=(700, 500))

figure[0,0].axes.visible = False

In [ ]:
def update_frame(p):
    """Update the frame using data received from the socket."""
    buff = get_buffer()
    if buff is not None:
        # Deserialize the buffer into a NumPy array
        data = np.frombuffer(buff, dtype=np.float64)

        data = data.reshape(20, 40)  

        if len(p.graphics) == 0:
            p.add_image(data, name="weights", cmap="viridis")
        else:
            # Update the line plot
            p["weights"].data = data
            p["weights"].cmap = gray

        p.auto_scale()

In [ ]:
# Add the animation update function
figure[0, 0].add_animations(update_frame)

figure.show()